# 01 - BaT Scraping

Scrape completed Porsche 911 auctions from Bring a Trailer.

**Outputs:**
- `data/raw/bat_listings.parquet` - Raw scraped data

In [1]:
# delete bad data if needed
# !rm data/raw/bat_listings.parquet

In [2]:
import logging
import pandas as pd
from pathlib import Path

from price_analysis.scraping import fetch_auctions, validate_scraped_data, DataQualityError
from price_analysis.scraping.bat import listings_to_dataframe

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Paths
DATA_DIR = Path("../data")
RAW_DIR = DATA_DIR / "raw"
RAW_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_PATH = RAW_DIR / "bat_listings.parquet"

## Configure Scraping

Adjust search queries and pagination as needed.

In [ ]:
# Search queries - multiple generation-specific searches to get more results
# BaT seems to limit how many results a single search returns
QUERIES = [
    # "Porsche 996",  # 1999-2004 (996.1 and 996.2)
    # "Porsche 997",  # 2005-2012 (997.1 and 997.2)
    "Porsche 991",  # 2012-2019 (991.1 and 991.2)
    "Porsche 992",  # 2020+ (992.1 and 992.2)
    "Porsche 911",  # Catch air-cooled + anything else missed
]

# Scraping parameters
MAX_CLICKS = 25  # Number of "Show More" button clicks (each loads ~24 listings)
DELAY = 1.0  # Seconds between requests (be polite!)
HEADLESS = True  # Set False to see browser for debugging

## Debug: Inspect BaT Page Structure (Optional)

Skip this section - fixture files already saved to `tests/fixtures/`.
Only re-run if BaT changes their DOM structure.

In [5]:
# Debug: Save sample pages for selector inspection
import time
from price_analysis.scraping import create_driver, save_debug_page

# Create driver (set HEADLESS=False above to watch)
driver = create_driver(headless=HEADLESS)

try:
    # 1. Fetch search results page
    search_url = "https://bringatrailer.com/auctions/results/?s=Porsche+911"
    logger.info(f"Fetching search page: {search_url}")
    driver.get(search_url)
    time.sleep(3)  # Wait for JS to load
    save_debug_page(driver, "bat_search_page", output_dir="../tests/fixtures")

    # 2. Fetch a specific listing page (992.1)
    listing_url = "https://bringatrailer.com/listing/2020-porsche-911-carrera-4s-coupe-26/"
    logger.info(f"Fetching listing: {listing_url}")
    driver.get(listing_url)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_992", output_dir="../tests/fixtures")

    # 3. Fetch another listing (997.2) for variety
    listing_url_997 = "https://bringatrailer.com/listing/2009-porsche-911-carrera-4s-coupe-28/"
    logger.info(f"Fetching listing: {listing_url_997}")
    driver.get(listing_url_997)
    time.sleep(3)
    save_debug_page(driver, "bat_listing_997", output_dir="../tests/fixtures")

    logger.info("Debug pages saved to tests/fixtures/")

finally:
    driver.quit()

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/flatljan/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver] found in cache
INFO:__main__:Fetching search page: https://bringatrailer.com/auctions/results/?s=Porsche+911
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_search_page.html
INFO:__main__:Fetching listing: https://bringatrailer.com/listing/2020-porsche-911-carrera-4s-coupe-26/
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_listing_992.html
INFO:__main__:Fetching listing: https://bringatrailer.com/listing/2009-porsche-911-carrera-4s-coupe-28/
INFO:price_analysis.scraping.bat:Saved debug page to ../tests/fixtures/bat_listing_997.html
INFO:__main__:Debug pages saved to tests/fixtures/


## Run Scraper

This will take a while depending on MAX_PAGES. Each page + listing takes ~3-5 seconds.

In [6]:
# Load existing URLs for incremental scraping (skip already-fetched listings)
existing_urls: set[str] = set()
if OUTPUT_PATH.exists():
    existing_df = pd.read_parquet(OUTPUT_PATH)
    existing_urls = set(existing_df["listing_url"])
    logger.info(f"Loaded {len(existing_urls)} existing URLs - will skip these")

all_listings = []

for query in QUERIES:
    logger.info(f"Scraping: {query}")
    listings = fetch_auctions(
        query=query,
        max_clicks=MAX_CLICKS,
        delay=DELAY,
        headless=HEADLESS,
        existing_urls=existing_urls,
    )
    all_listings.extend(listings)
    logger.info(f"Found {len(listings)} NEW listings for '{query}'")

logger.info(f"Total NEW listings scraped: {len(all_listings)}")

INFO:__main__:Loaded 818 existing URLs - will skip these
INFO:__main__:Scraping: Porsche 991
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/flatljan/.wdm/drivers/chromedriver/mac64/143.0.7499.169/chromedriver-mac-arm64/chromedriver] found in cache
INFO:price_analysis.scraping.bat:Fetching search results: https://bringatrailer.com/auctions/results/?s=Porsche+991
INFO:price_analysis.scraping.bat:Clicked 'Show More' (1/5)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (2/5)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (3/5)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (4/5)
INFO:price_analysis.scraping.bat:Clicked 'Show More' (5/5)
INFO:price_analysis.scraping.bat:Found completed auctions container
INFO:price_analysis.scraping.bat:Found 144 completed auction cards
INFO:price_analysis.scraping.bat:Skipped 14 unsold listings (bid to)
I

## Convert to DataFrame and Save

In [7]:
df = listings_to_dataframe(all_listings)

# Run data quality checks and filter low-price listings (<$10k)
# Returns filtered DataFrame, raises DataQualityError if checks fail
df = validate_scraped_data(df)

display(df.head(10))
print(f"\nShape: {df.shape}")

INFO:price_analysis.scraping.bat:Filtered 38 listings below $10,000 (likely parts/salvage)
INFO:price_analysis.scraping.bat:Data quality checks passed: 256 listings, 256 with prices


,listing_url,title_raw,sale_price,sale_date,model_year,generation,trim,transmission,mileage,color,location
1,https://bringatrailer.com/listing/2014-porsche...,31k-Mile 2014 Porsche 911 Carrera 4S Coupe,83500,2025-09-27,2014.0,991.1,Carrera 4S,PDK,31000.0,Black,"Santa Monica, California 90405"
2,https://bringatrailer.com/listing/2015-porsche...,37k-Mile 2015 Porsche 911 Carrera 4S Coupe,82500,2025-11-30,2015.0,991.1,Carrera 4S,PDK,37000.0,Agate Gray Metallic,"Miami, Florida 33161"
3,https://bringatrailer.com/listing/2014-porsche...,23k-Mile 2014 Porsche 911 50th Anniversary Edi...,134500,2025-11-26,2014.0,991.1,None,PDK,23000.0,Graphite Gray,"Seattle, Washington 98115"
4,https://bringatrailer.com/listing/2014-porsche...,2014 Porsche 911 GT3,115000,2025-11-21,2014.0,991.1,GT3,PDK,36000.0,White,"Maryville, Tennessee 37804"
5,https://bringatrailer.com/listing/2016-porsche...,16k-Mile 2016 Porsche 911 Carrera GTS Coupe,129000,2025-09-26,2016.0,991.2,Carrera,PDK,16000.0,Sapphire Blue Metallic,"Phoenix, Arizona 85018"
6,https://bringatrailer.com/listing/brakes-4/,Porsche 991 Ceramic Composite Brakes,13750,2025-10-29,NaN,None,None,None,NaN,Yellow & Black Porsche Script,"Chicago, Illinois 60642"
7,https://bringatrailer.com/listing/2019-porsche...,2019 Porsche 911 GT3 RS,190000,2025-12-16,2019.0,991.2,GT3 RS,PDK,44000.0,White,"Plano, Texas 75093"
8,https://bringatrailer.com/listing/2017-porsche...,Azurro California Blue 2017 Porsche 911 Turbo ...,166500,2025-10-29,2017.0,991.2,Turbo S,PDK,20000.0,PTS Azurro California Blue,"Carthage, North Carolina 28327"
10,https://bringatrailer.com/listing/2013-porsche...,37k-Mile 2013 Porsche 911 Carrera 4S Coupe,62750,2025-10-10,2013.0,991.1,Carrera 4S,PDK,37000.0,Platinum Silver Metallic,"North Salem, New York 10560"
11,https://bringatrailer.com/listing/2019-porsche...,2019 Porsche 911 GT3 Cup,145000,2025-09-26,2019.0,991.2,GT3,None,10000.0,None,"Escondido, California 92029"



Shape: (256, 11)


In [8]:
# Append to existing if present, otherwise create new
if OUTPUT_PATH.exists():
    existing = pd.read_parquet(OUTPUT_PATH)
    df = pd.concat([existing, df], ignore_index=True)
    df = df.drop_duplicates(subset=["listing_url"], keep="last")
    logger.info(f"Merged with existing data: {len(df)} total listings")

df.to_parquet(OUTPUT_PATH, index=False)
logger.info(f"Saved to {OUTPUT_PATH}")

INFO:__main__:Merged with existing data: 1053 total listings
INFO:__main__:Saved to ../data/raw/bat_listings.parquet


## Quick Inspection

In [9]:
print("Counts by generation:")
display(df["generation"].value_counts(dropna=False))

Counts by generation:


generation
997.1    273
996.1    186
996.2    169
997.2    147
991.2     71
None      71
992.1     58
991.1     43
992.2     35
Name: count, dtype: int64

In [10]:
print("Counts by trim:")
display(df["trim"].value_counts(dropna=False))

Counts by trim:


trim
Carrera        271
Turbo          165
Carrera S      138
Carrera 4S     133
None            73
Carrera 4       67
GT3             57
Turbo S         51
GT3 RS          27
Targa           19
GT3 Touring     17
Targa 4S        17
Targa 4         13
GT2 RS           5
Name: count, dtype: int64

In [11]:
print("Counts by transmission:")
display(df["transmission"].value_counts(dropna=False))

Counts by transmission:


transmission
Manual       721
PDK          197
Tiptronic     50
None          43
Automatic     42
Name: count, dtype: int64

In [12]:
# Check parsing quality - how many have all required fields?
required = ["sale_price", "model_year", "generation", "trim", "transmission", "mileage"]
complete = df[required].notna().all(axis=1).sum()
print(
    f"\nListings with all required fields: {complete} / {len(df)} ({complete / len(df) * 100:.1f}%)"
)


Listings with all required fields: 929 / 1053 (88.2%)


In [13]:
# Sample some listings to verify parsing
print("Sample listings for manual verification:")
sample = df.sample(min(10, len(df)), random_state=42)
for _, row in sample.iterrows():
    print(f"\n{row['title_raw']}")
    print(
        f"  Parsed: {row['model_year']} {row['generation']} {row['trim']} ({row['transmission']})"
    )
    print(f"  Price: ${row['sale_price']:,}" if pd.notna(row["sale_price"]) else "  Price: N/A")
    print(f"  Mileage: {row['mileage']:,}" if pd.notna(row["mileage"]) else "  Mileage: N/A")

Sample listings for manual verification:

2007 Porsche 911 Carrera Coupe 6-Speed
  Parsed: 2007.0 997.1 Carrera (Manual)
  Price: $47,000
  Mileage: 63,000.0

33k-Mile 2002 Porsche 911 Turbo Coupe X50 6-Speed
  Parsed: 2002.0 996.2 Turbo (Manual)
  Price: $90,996
  Mileage: 33,000.0

2022 Porsche 911 GT3 Touring 6-Speed
  Parsed: 2022.0 992.1 GT3 Touring (Manual)
  Price: $258,008
  Mileage: 6,000.0

1987 Porsche 911 Carrera Targa G50
  Parsed: 1987.0 None Targa (Manual)
  Price: $91,611
  Mileage: 50,000.0

32k-Mile 2003 Porsche 911 Carrera 4S Coupe 6-Speed
  Parsed: 2003.0 996.2 Carrera 4S (Manual)
  Price: $37,750
  Mileage: 32,000.0

1999 Porsche 911 Carrera Cabriolet 6-Speed
  Parsed: 1999.0 996.1 Carrera (Manual)
  Price: $25,000
  Mileage: 67,000.0

1999 Porsche 911 Carrera Coupe 6-Speed
  Parsed: 1999.0 996.1 Carrera (Manual)
  Price: $19,996
  Mileage: 131,000.0

1999 Porsche 911 Carrera 4 Coupe
  Parsed: 1999.0 996.1 Carrera 4 (Manual)
  Price: $27,996
  Mileage: 94,000.0

20